In [13]:
import pandas as pd
import numpy as np
import math as m

In [70]:
data=pd.read_csv('data2.csv')
data_T=pd.DataFrame(data, columns= ['Object','Property','Value','Source'])
data_T

Object   Property   Value    Source
0         Object1  Property5  287225   Source1
1         Object1  Property6  304071   Source1
2         Object2  Property5  438474   Source1
3         Object2  Property6  288522   Source1
4         Object3  Property5  354622   Source1
...           ...        ...     ...       ...
59995   Object996  Property4  846054  Source10
59996   Object997  Property4  577352  Source10
59997   Object998  Property4  871969  Source10
59998   Object999  Property4  768953  Source10
59999  Object1000  Property4  944904  Source10

[60000 rows x 4 columns]

In [71]:
data_T['TSInitial']=0.8
data_T['C_v']=0.0
data_T.head(5)

Object   Property   Value   Source  TSInitial  C_v
0  Object1  Property5  287225  Source1        0.8  0.0
1  Object1  Property6  304071  Source1        0.8  0.0
2  Object2  Property5  438474  Source1        0.8  0.0
3  Object2  Property6  288522  Source1        0.8  0.0
4  Object3  Property5  354622  Source1        0.8  0.0

In [72]:
# Definition de la matice de similarite
def similarity(dataframe):
    dict_sim = {}
    for key, df in dataframe.groupby(by=['Object','Property']):
        Values = df['Value'].unique()
        row = key[0]+key[1]
        for  i in range(len(Values)):
            w1 = Values[i]
            for  u in range(len(Values)):
                w2 = Values[u]
                sim = 1
                if w1!=w2:
                    t = abs(w1-w2)
                    sim = 1/t
                dict_sim[row+str(w1)+str(w2)] = sim
                dict_sim[row+str(w2)+str(w1)] = sim
    return dict_sim

In [73]:
import math
def Ts(df):
    ts_list = df['TSInitial'].to_list()
    return sum([-math.log(1-ts) for ts in ts_list])
def cos_sim(a,b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [74]:
import time as t
from natsort import index_natsorted, order_by_index
def TruthFinder_seq(data_T):
    ni=5
    trustold = 0.00001
    start = t.time()
    nb_iter_do = 0
    rho=0.7
    lam=0.5
    dict_sim=similarity(data_T)
    for i in range(ni):
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts1=df['TSInitial']
        grouped_df = data_T.groupby(["Object", "Property","Value"])
        for key,item in grouped_df:
            sum_ = Ts(item)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=sum_
        grouped_df=data_T.groupby(["Object", "Property","Value"])
        for key ,item in grouped_df:
            cv=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"].values[0]
#             print(cv)

            data_d=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]!=key[2])), ["Value","C_v"]]
            for row in data_d.values:
                Key_sim=key[0]+key[1]+str(key[2])+str(int(row[0]))
                cv+=rho*(row[1]*dict_sim[Key_sim])
            cv=1/(1+np.exp(-lam*cv))
        #     print(cv)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=cv
        groupe_ts=data_T.groupby(['Source'])
        for key,item in groupe_ts:
            ts_=item['C_v'].mean()
            data_T.loc[((data_T["Source"] == key)), "TSInitial"]=ts_
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts2=df['TSInitial']
        print('Itération : ',i+1)
        print(1-abs(cos_sim(ts1,ts2)))
        nb_iter_do = i+1
        if 1-abs(cos_sim(ts1,ts2)) < trustold:
            break
    end = t.time()

    return data_T,end-start,nb_iter_do



In [ ]:
result_=TruthFinder_seq(data_T)

In [44]:
result_

(         Object   Property   Value   Source  TSInitial       C_v
 0       Object1  Property3  418886  Source1   0.777107  0.903040
 1       Object1  Property2  354490  Source1   0.777107  0.903040
 2       Object2  Property3  225299  Source1   0.777107  0.903040
 3       Object2  Property2  359195  Source1   0.777107  0.903040
 4       Object3  Property3  345497  Source1   0.777107  0.903040
 ...         ...        ...     ...      ...        ...       ...
 1495   Object96  Property1  788646  Source4   0.689528  0.644761
 1496   Object97  Property1  766439  Source4   0.689528  0.644702
 1497   Object98  Property1  607933  Source4   0.689528  0.644708
 1498   Object99  Property1  869887  Source4   0.689528  0.786084
 1499  Object100  Property1  805964  Source4   0.689528  0.794360
 
 [1500 rows x 6 columns],
 18.15455985069275,
 4)

In [45]:
def res(result):
    gr=result.groupby(["Object", "Property"])
    data_out={'Object':[],'Property':[],'Value':[]}
    for key,item in gr:
#         print(item.values)
        cv_max=0
        value_pred=None
        for line in item.values:
#             print(line[5])
            if cv_max<line[5]:
                cv_max=line[5]
                value_pred=line[2]
        data_out['Object'].append(key[0])
        data_out['Property'].append(key[1])
        data_out['Value'].append(value_pred)
    return pd.DataFrame(data_out)
        
        

In [46]:
res_=res(result_[0])
# len(res_)

In [47]:
list_ = res_.to_numpy()

In [50]:
data_truth=pd.read_csv('data_truth1.csv')
data_truth_=pd.DataFrame(data_truth, columns= ['Object','Property','Value'])


In [52]:
len(data_truth_)

300

In [53]:
list__=data_T.to_numpy()
# list__

In [54]:
list_1=data_truth_.to_numpy()


In [55]:
# def function(list_A,list_B):
#     binary=[]
#     search_list = [tuple(li[:-3]) for li in list_B]
#     for ser_item in search_list:
#         if ser_item in list_A:
#             binary.append(1)
#         else:
#             binary.append(0)
#     return binary

In [56]:
def function(list_A,list_B):
    binary=[]
    search_list = [tuple(li[:-3]) for li in list_B]
    list_A = [tuple(li) for li in list_A]
    for ser_item in search_list:
        if ser_item in list_A:
            binary.append(1)
        else:
            binary.append(0)
    return binary

In [57]:
res = function(list_, list__)
res_truth=function(list_1, list__)
# res
# res_truth

In [58]:
# !pip install scikit-learn
import sklearn

In [59]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [60]:
precision_score(res_truth,res)

0.8243451463790447

In [61]:
accuracy_score(res_truth,res)

0.8633333333333333

In [62]:
recall_score(res_truth,res)

0.854632587859425

In [63]:
f1_score(res_truth,res)

0.8392156862745097